In [63]:
## https://planetarycomputer.microsoft.com/dataset/noaa-nclimgrid-monthly

import os, os.path
from pystac_client import Client
import planetary_computer as pc
import rioxarray
import rasterio as rio
import fiona
import pyproj
from shapely.geometry import shape, Point, Polygon, mapping
from functools import partial
from shapely.ops import transform
import urllib.request
import numpy as np
import geopandas as gpd
import pyproj
import rich
# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
pc.settings.set_subscription_key("st=2022-06-01T22%3A55%3A32Z&se=2022-06-02T23%3A40%3A32Z&sp=rl&sv=2020-06-12&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2022-06-02T20%3A23%3A33Z&ske=2022-06-09T20%3A23%3A33Z&sks=b&skv=2020-06-12&sig=U5e1SoZGZqIcunum5BKWcmsVi2bmS1tBsr94LPI%2BLoc%3D")

In [52]:
import pystac_client
import planetary_computer

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [53]:
import fiona

shpfile = 'data/city_limit_4326.shp'

lyr = fiona.open(shpfile)
for feat in lyr:
    area_of_interest_city = feat['geometry']
    # print(area_of_interest)

left, bottom, right, top = lyr.bounds
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [left, bottom],
            [right, bottom],
            [right, top],
            [left, top],
            [left, bottom],
        ]
    ],
}

print(area_of_interest)

{'type': 'Polygon', 'coordinates': [[[-75.28030313034645, 39.867465570687145], [-74.9557457320632, 39.867465570687145], [-74.9557457320632, 40.137927528193686], [-75.28030313034645, 40.137927528193686], [-75.28030313034645, 39.867465570687145]]]}


In [54]:
## check on GEE, https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ
range_time = '2021-03-01' + '/' + '2021-10-31'

search_new = catalog.search(
    collections=["noaa-nclimgrid-monthly"], intersects=area_of_interest, datetime=range_time
)

items_tiles = search_new.item_collection()

print(f"{len(items_tiles)} Items found in the 'new' range")

8 Items found in the 'new' range


In [56]:
outfolder = 'climate-monthly'
if not os.path.exists(outfolder): os.mkdir(outfolder)

## loop all the intersected tiles and download them all
for item in items_tiles:
    #href = pc.sign(item.assets["image"].href)
    href = item.assets["tavg"].href
    print(href)
    
    outfilename = os.path.join(outfolder, item.id + ".tif")
    urllib.request.urlretrieve(href, outfilename)

https://nclimgridwesteurope.blob.core.windows.net/nclimgrid-cogs/nclimgrid-monthly/nclimgrid-tavg-202110.tif?st=2024-02-23T03%3A21%3A55Z&se=2024-02-24T04%3A06%3A55Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2024-02-24T03%3A21%3A54Z&ske=2024-03-02T03%3A21%3A54Z&sks=b&skv=2021-06-08&sig=vjzen7Sl7hfnLWFr3CS12iR%2BqTo/UmxyLx7Ky7Nt4lA%3D
https://nclimgridwesteurope.blob.core.windows.net/nclimgrid-cogs/nclimgrid-monthly/nclimgrid-tavg-202109.tif?st=2024-02-23T03%3A21%3A55Z&se=2024-02-24T04%3A06%3A55Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2024-02-24T03%3A21%3A54Z&ske=2024-03-02T03%3A21%3A54Z&sks=b&skv=2021-06-08&sig=vjzen7Sl7hfnLWFr3CS12iR%2BqTo/UmxyLx7Ky7Nt4lA%3D
https://nclimgridwesteurope.blob.core.windows.net/nclimgrid-cogs/nclimgrid-monthly/nclimgrid-tavg-202108.tif?st=2024-02-23T03%3A21%3A55Z&se=2024-02-24T04%3A06%3A55Z&sp=rl&sv=2021-06-08